In [1]:
import pandas as pd
import numpy as np
import datetime
import os

os.getcwd()

'/home/jian/Projects/Saatva/MMM/URL_models/input'

In [2]:
sales_data=pd.read_csv('/home/jian/Projects/Saatva/MMM/Input/sales_data.csv',dtype=str)
sales_data_saatva=sales_data[sales_data['brand']=='saatva']
sales_data_saatva['sales']=sales_data_saatva['sales'].astype(float).round(2)

sales_data_saatva['date']=sales_data_saatva['date_time'].apply(lambda x: datetime.datetime.strptime(x.split(" ")[0],"%Y-%m-%d").date())
sales_data_2018_Saatva=sales_data_saatva[sales_data_saatva['date'].apply(lambda x: x.year==2018)]
sales_data_2018_Saatva['date']=sales_data_2018_Saatva['date'].astype(str)
sales_data_2018_Saatva_sales=sales_data_2018_Saatva.groupby(['date'])['sales'].sum().to_frame().reset_index().rename(columns={"sales":"sales_net"})
sales_data_2018_Saatva_trans=sales_data_2018_Saatva.groupby(['date'])['sales'].count().to_frame().reset_index().rename(columns={"sales":"trans_net"})
sales_data_2018_Saatva=pd.merge(sales_data_2018_Saatva_sales,sales_data_2018_Saatva_trans,on="date")
sales_data_2018_Saatva.shape

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

(365, 3)

In [3]:
SM_input_excel=pd.ExcelFile("/home/jian/Projects/Saatva/MMM/URL_models/input/sm_all_urls_0417.xlsx")
SM_input_excel.sheet_names

['Home Page', 'Blogs', 'Other', '404 Pages']

In [4]:
sales_data_2018_Saatva.head(2)

,date,sales_net,trans_net
0,2018-01-01,330007.80,232
1,2018-01-02,316132.57,227


In [6]:
agg_SM_data=pd.DataFrame()
append_trans_df_list=[]

writer=pd.ExcelWriter("/home/jian/Projects/Saatva/MMM/URL_models/input/URL_SM_input_tabs_data_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
for sheet in SM_input_excel.sheet_names:
    
    df=SM_input_excel.parse(sheet)
    df_date=df.rename(columns={"Date":"date"})
    locals()['df_'+sheet.replace(" ","")]=pd.merge(sales_data_2018_Saatva,df_date,on='date')
    locals()['df_'+sheet.replace(" ","")].to_excel(writer,sheet,index=False)
    locals()['df_'+sheet.replace(" ","")].describe().T.to_excel(writer,"Stat_"+sheet.replace(" ",""),index=True)
    
    
    for col in df.columns.tolist():
        if col not in ["Date",'Users','Sessions','ga:goal11Completions','Time on Page','Transactions']:
            df=df.rename(columns={col:sheet.replace(" ","")+"|"+col})
    if len(agg_SM_data)==0:
        agg_SM_data=df
    else:
        agg_SM_data=pd.merge(agg_SM_data,df,on=["Date",'Users','Sessions','ga:goal11Completions','Time on Page','Transactions'])
agg_SM_data=agg_SM_data.rename(columns={"Date":"date"})

    
writer.save()   

In [7]:
agg_SM_data=pd.merge(sales_data_2018_Saatva,agg_SM_data,on="date")


In [8]:
agg_SM_data.to_csv("/home/jian/Projects/Saatva/MMM/URL_models/input/URL_SM_input_data_JL_"+str(datetime.datetime.now().date())+".csv",index=False)